In [53]:
# ! pip install tiktoken --quiet
# ! pip install openai --quiet
# ! pip install python-dotenv --quiet

In [59]:
import pandas as pd
import json
import tiktoken
from dotenv import load_dotenv
from openai import OpenAI

In [56]:
load_dotenv()

True

In [27]:
df = pd.read_csv('../data/amazon-reviews-for-2-cats.csv')

In [28]:
df.head()

,asin,reviews,total_reviews,first_review_date,last_review_date,helpful_votes,verified_purchase,category
0,B00006IUTN,"[{""review_title"": ""Five Stars"", ""review_text"":...",509,2006-08-30 14:33:21.000,2023-03-20 17:28:03.486,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, ...","[true, true, true, true, true, true, true, tru...",1
1,B00006IUV3,"[{""review_title"": ""Two Stars"", ""review_text"": ...",722,2004-03-14 16:51:23.000,2023-05-18 01:54:55.930,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[true, false, true, true, true, true, true, tr...",1
2,B00009W3G4,"[{""review_title"": ""Very disappointed"", ""review...",896,2007-10-28 07:01:11.000,2023-02-22 05:10:19.429,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 4, 0, 0, 0, ...","[true, true, true, true, true, true, true, tru...",1
3,B00009W3HD,"[{""review_title"": ""great"", ""review_text"": ""wor...",771,2007-01-22 12:21:40.000,2023-05-08 18:13:00.943,"[0, 0, 0, 2, 0, 0, 3, 0, 7, 0, 0, 0, 5, 0, 0, ...","[true, true, true, true, true, true, true, tru...",1
4,B00009W3I4,"[{""review_title"": ""Great vent!"", ""review_text""...",1037,2007-08-03 16:43:11.000,2023-05-17 17:38:36.885,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 4, ...","[true, true, true, true, true, true, true, tru...",1


In [30]:
df['reviews'] = df['reviews'].apply(json.loads)
df['helpful_votes'] = df['helpful_votes'].apply(json.loads)
df['verified_purchase'] = df['verified_purchase'].apply(json.loads)

In [34]:
print(type(df['reviews'][0]))

<class 'list'>


In [ ]:
#B00006IUTN - asin
product1_reviews = str(df['reviews'][0])


In [75]:
product1_reviews = product1_reviews[:-500]

In [76]:
product1_reviews

'[{\'review_title\': \'Five Stars\', \'review_text\': \'Just get it!\'}, {\'review_title\': \'They do a great job and make it easy to dump out the ...\', \'review_text\': \'I use these every day in a single serve coffeepot.  I get them in a package of 12 boxes (with each box containing 40 filters).  They do a great job and make it easy to dump out the used grounds.  While my coffeemaker does not require that I use these, it is so much easier and neater that I always use them anyway.\'}, {\'review_title\': \'Great for one-shot coffeemaker\', \'review_text\': \'These filters are hard to find in stores and I have reordered them a couple of times from Amazon.  They make the coffee taste better, help with the cleanup and keep coffee grounds from falling into your cup.  Great idea!\'}, {\'review_title\': \'and they are very good quality, and unbleached\', \'review_text\': \'Cannot find this size filter in any of the major food store chains, nor in the shopping supercenter! Thank goodness for

# Estimate Embedding

In [57]:
# encoding = tiktoken.encoding_for_model("gpt-4o")
# ints = encoding.encode(product1_reviews)
# print(len(ints))


28571


# Get Embedding

In [62]:
client = OpenAI()

def split_text_by_words(text, max_words=1000):
    text = text.replace("\n" , " ")
    words = text.split()  # Split text into individual words
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i:i + max_words])  # Combine words back into a chunk
        chunks.append(chunk)
    return chunks

# Split text into chunks of 1000 words
chunks = split_text_by_words(product1_reviews, max_words=1000)

# Generate embeddings for each chunk
embeddings = []
for chunk in chunks:
    emb = client.embeddings.create(
        input=[chunk],
        model="text-embedding-3-large"
    )
    embeddings.append(emb)


In [63]:
product_1_reviews_embedding = embeddings

In [64]:
product_1_reviews_embedding

[CreateEmbeddingResponse(data=[Embedding(embedding=[0.040023304522037506, -0.008806626312434673, -0.003668803023174405, 0.02767154388129711, -0.020476343110203743, -0.00013936017057858407, 0.0018803083803504705, -0.009391236118972301, -0.008394400589168072, 0.03321784362196922, 0.02917054295539856, -0.024913383647799492, -0.024193862453103065, -0.02719186246395111, 0.01275649107992649, 0.005377663299441338, -0.006239588372409344, -0.015754491090774536, 0.015424711629748344, 0.021540632471442223, -0.021390732377767563, -0.0025857752189040184, -0.031538963317871094, 0.037235163152217865, 0.04919718578457832, 0.016953691840171814, -0.02194536291062832, -0.00107459572609514, -0.004129745531827211, -0.03750498592853546, 0.018992332741618156, 0.02305462211370468, -0.002664472907781601, 0.022634902969002724, -0.03330778330564499, -0.004242170602083206, 0.002121085301041603, 0.012216851115226746, -0.007064038421958685, 0.0295003242790699, 0.011887071654200554, -0.023759152740240097, -0.0171785

In [66]:
def chat_completion(final_prompt):
    completion = client.chat.completions.create(
        model = 'gpt-4o',
        messages=[
            {"role":"user","content":final_prompt}
        ],
        temperature=0
    )
    return completion.choices[0].message.content

In [77]:
prompt = f"""

“Read and analyze more than 500 user reviews about a product. Identify and summarize the key insights under the following categories:
	1.	Overall Sentiment: The general tone of the reviews (positive, negative, or mixed).
	2.	Common Pros: Highlight the most appreciated features or benefits mentioned by users.
	3.	Common Cons: Summarize the most frequently reported issues or complaints.
	4.	Key Trends: Any recurring themes or patterns across the reviews.
	5.	Suggestions/Improvements: Suggestions provided by users for improving the product.

Ensure the summary is concise yet comprehensive, and include specific examples or phrases from the reviews to support the analysis where relevant.”

```
{product1_reviews}
```

"""

In [79]:
response = chat_completion(prompt)

In [81]:
import pprint
pprint.pprint(response)

('Based on the analysis of over 500 user reviews for the Melitta #1 Cone '
 'Coffee Filters, here is a summary of the key insights:\n'
 '\n'
 '1. **Overall Sentiment**: \n'
 '   - The general sentiment of the reviews is predominantly positive. Many '
 'users express satisfaction with the product, particularly appreciating its '
 'functionality and availability through Amazon. However, there are a few '
 'negative reviews, primarily related to sizing issues and misleading product '
 'descriptions.\n'
 '\n'
 '2. **Common Pros**:\n'
 '   - **Availability**: Many users appreciate that these filters are '
 'available on Amazon, as they are hard to find in local stores.\n'
 '   - **Quality and Performance**: Users frequently mention the high quality '
 'of the filters, noting that they effectively prevent coffee grounds from '
 'entering the cup and improve the taste of the coffee.\n'
 '   - **Environmental Benefits**: The unbleached, natural brown filters are '
 'praised for being environme